# Лабораторная работа №8 Генерация текста на основе “Алисы в стране чудес”
### Выполнила студентка группы БВТ2101 Пьянова Анна Олеговна

### Цель работы
Рекуррентные нейронные сети также могут быть использованы в качестве генеративных
моделей.
Это означает, что в дополнение к тому, что они используются для прогнозных моделей
(создания прогнозов), они могут изучать последовательности проблемы, а затем
генерировать совершенно новые вероятные последовательности для проблемной
области.
Подобные генеративные модели полезны не только для изучения того, насколько хорошо
модель выявила проблему, но и для того, чтобы узнать больше о самой проблемной
области.

### Задачи:
- Ознакомиться с генерацией текста
- Ознакомиться с системой Callback в Keras


### Выполнение работы

In [1]:
import sys
import datetime
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input
from keras.callbacks import ModelCheckpoint, Callback, TensorBoard
from tensorflow.keras.utils import to_categorical

Загрузка текста книги, подготовка данных для моделирования

In [2]:
filename = "wonderland.txt"
with open(filename, 'r', encoding='utf-8') as file:
    raw_text = file.read().lower()

chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  144679
Total Vocab:  51


Конвертация символов в целые числа с помощью ранее подготовленной таблицы поиска

In [3]:
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
    
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)

Total Patterns:  144579


Преобразование списка входных последовательностей для использования с Keras

In [4]:
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

Определение LSTM модели

In [6]:
model = Sequential()
model.add(Input(shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.keras"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [12]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - loss: 3.0965
Epoch 1: loss improved from inf to 3.01685, saving model to weights-improvement-01-3.0168.keras
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 265s 233ms/step - loss: 3.0964
Epoch 2/20
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 245ms/step - loss: 2.8506
Epoch 2: loss improved from 3.01685 to 2.81757, saving model to weights-improvement-02-2.8176.keras
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 277s 245ms/step - loss: 2.8506
Epoch 3/20
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 216ms/step - loss: 2.7241
Epoch 3: loss improved from 2.81757 to 2.70970, saving model to weights-improvement-03-2.7097.keras
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 244s 216ms/step - loss: 2.7240
Epoch 4/20
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 2.6532
Epoch 4: loss improved from 2.70970 to 2.63916, saving model to weights-improvement-04-2.6392.keras
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 231s 204ms/step - loss: 2.6532
Epoch 5/20
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step - loss: 2.5838
E

Генерация текста с помощью сети LSTM

In [5]:
model = Sequential()
model.add(Input(shape=(X.shape[1], X.shape[2])))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "weights-improvement-20-2.0076.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [7]:
# pick a random seed
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

Seed:
" ourt_.”

everybody looked at alice.

“_i’m_ not a mile high,” said alice.

“you are,” said the king. "
 “ih io would toe koot be anoine to the toons!”

“h whou d shrl mo his to ”hur mote ” thiught alice, “ih wou’te you dno the horse to the karee hare ” 
“h den’t teon toe thing?” she mock turtle seilied 
the wai io a ger lece to the tab it an an anoed toe tai  “ho  yhu  i wouldn the korse woule ”ou goow the dormo, io a lorg aro oi the sooe.”

“i whilk y said the mock turtle, “ih io wish the woudd ae a loeg tu the thate  and i sas an toed an io ”h
the koot an a lors ”ou shonkn the mooer.” 
“i whin you  a  oh _ourse,” thiught alice, “in wou dn wou toonte toe thin ho ”h
thenk io would be a loog aro oi atery_nnn, and it would be a loog tiie  she wisle bate wai iott an an anl of the sabbit, and the woide to the gurhous woice  the was tointing at the could  the was tointing an the courd  ao ae ano whthe to thyh the soee  the was to toint toen a cond oide toee and the card so tee thet 

Реализация собственного CallBack

In [ ]:
class TextGenerationCallback(Callback):
    def __init__(self, dataX, int_to_char, n_vocab, seq_length):
        super(TextGenerationCallback, self).__init__()
        self.dataX = dataX
        self.int_to_char = int_to_char
        self.n_vocab = n_vocab
        self.seq_length = seq_length

    def on_epoch_end(self, epoch, logs=None):
        start = np.random.randint(0, len(self.dataX) - 1)
        pattern = self.dataX[start]
        print(f"\nEpoch {epoch + 1} - Generated Text:")
        print("Seed:")
        print("\"", ''.join([self.int_to_char[value] for value in pattern]), "\"")
        # generate characters
        for i in range(100):
            x = np.reshape(pattern, (1, len(pattern), 1))
            x = x / float(self.n_vocab)
            prediction = self.model.predict(x, verbose=0)
            index = np.argmax(prediction)
            result = self.int_to_char[index]
            seq_in = [self.int_to_char[value] for value in pattern]
            sys.stdout.write(result)
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
        print("\n")

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

text_gen_callback = TextGenerationCallback(dataX, int_to_char, n_vocab, seq_length)

model.fit(X, y, epochs=10, batch_size=128, callbacks=[text_gen_callback, tensorboard_callback])


Epoch 1/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 186ms/step - loss: 1.9741
Epoch 1 - Generated Text:
Seed:
" lice, she went on, “what’s your name, child?”

“my name is alice, so please your majesty,” said alic "
e in a soneoo tonc. 
“ih coorse toeh io ”hur a tey oo tie,” said the guyphon, “io would toe kocw tha

1130/1130 ━━━━━━━━━━━━━━━━━━━━ 218s 191ms/step - loss: 1.9741
Epoch 2/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 1.9440
Epoch 2 - Generated Text:
Seed:
" e i’m not ada,” she said, “for her hair goes in such long
ringlets, and mine doesn’t go in ringlets  "
an anl  a donro taa int tee sooe of the sab it an in was toier in the ras. 
“h woolt toen to ae anda

1130/1130 ━━━━━━━━━━━━━━━━━━━━ 184s 163ms/step - loss: 1.9440
Epoch 3/10
1130/1130 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 1.9228
Epoch 3 - Generated Text:
Seed:
" . (it was
this last remark that had made the whole party look so grave and
anxious.)

alice could th "
e katter was a little broree  she cade to tie ta

In [8]:
filename = "model.keras"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = np.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:]
    
print("\nDone.")

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 12 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Seed:
"  she had
drunk half the bottle, she found her head pressing against the ceiling,
and had to stoop to "
 toen the roee afner her ane ano ofc thet, and she was soeiting at the could sed whn, and tee whnt on anoneed anonh the carese and she whnee har hne the ragl of the saali. 
“he you don’t know what _ yhre then you toon lo ”our majesty,” said the mrek turtle. 
“ie course wour have ”our majesty,” said the manch hare.

“ie course oo hes seme” an i meoe,” said the dat, “io you dne’ wesh the dorttr of the bance.
int ier weil then!”

“ho  yher _s _ lore waye” said theee ”hur mace ”ou sorl oo here to toaa and oucered th the white ” she said to herself, “it would be anhan  a dondo tfll the wond,”

“h whsh y said the mucen so the cormerse in the wan oo ano oiretee. 
“ie you doe’ was in anl the seal” ”hat i toudd woul here a gen in ”hs aaner? ii a large fande?” 
“i whal the was no ” said the mock turtle. 
“ie course wour have ”our majesty,” said the manch hare.

“ie course mo tee,” said 

Отслеживание процесса обучения с помощью TensorBoard

In [9]:
%load_ext tensorboard
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6007 (pid 15056), started 11:32:00 ago. (Use '!kill 15056' to kill it.)

In [18]:
model.save_weights('model.weights.h5')
model.save('model.keras')